In [1]:
import dvc.api
import pandas as pd

In [2]:
df = pd.read_csv("..\data\creditcard.csv")

In [3]:
data_unseen = df.sample(frac=0.1, random_state=42)
df = df.drop(data_unseen.index)
df = df.drop(columns=["Time"])

display(df.head())

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0


In [4]:
from pycaret.classification import *

In [5]:
expt1 = setup(
data = df,
target = "Class",
session_id = 42,
train_size = 0.8,
normalize = True,
fix_imbalance = True,
)

,Description,Value
0,session_id,42
1,Target,Class
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(256326, 30)"
5,Missing Values,False
6,Numeric Features,29
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [6]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [7]:
top5 = compare_models(sort="F1", fold = 5, n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9995,0.9706,0.8072,0.8789,0.8414,0.8411,0.8420,18.5760
rf,Random Forest Classifier,0.9994,0.9614,0.7959,0.8386,0.8167,0.8164,0.8167,60.4100
lightgbm,Light Gradient Boosting Machine,0.9984,0.9611,0.8186,0.5277,0.6388,0.6381,0.6550,3.2280
knn,K Neighbors Classifier,0.9978,0.9243,0.8327,0.4275,0.5647,0.5637,0.5956,146.8700
dt,Decision Tree Classifier,0.9972,0.8727,0.7478,0.3535,0.4793,0.4780,0.5125,10.5140
gbc,Gradient Boosting Classifier,0.9876,0.9737,0.8726,0.1123,0.1985,0.1960,0.3095,127.4940
ridge,Ridge Classifier,0.9877,0.0000,0.8243,0.1067,0.1888,0.1863,0.2937,0.3200
lda,Linear Discriminant Analysis,0.9877,0.9665,0.8243,0.1067,0.1888,0.1863,0.2937,1.3260
ada,Ada Boost Classifier,0.9771,0.9657,0.8896,0.0644,0.1199,0.1171,0.2352,27.8600
nb,Naive Bayes,0.9762,0.9538,0.8497,0.0587,0.1097,0.1069,0.2196,0.4200


In [8]:
for model in top5:
    print(model)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=42, verbose=0,
                     warm_start=False)
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False

In [9]:
et = top5[0]

tuned_et = tune_model(et, fold=5, optimize="F1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9988,0.9805,0.7429,0.6341,0.6842,0.6836,0.6858
1,0.9989,0.9597,0.8286,0.6304,0.7160,0.7155,0.7222
2,0.9990,0.9672,0.8169,0.6824,0.7436,0.7431,0.7461
3,0.9992,0.9858,0.9014,0.7273,0.8050,0.8047,0.8093
4,0.9992,0.9699,0.8169,0.7532,0.7838,0.7834,0.7840
Mean,0.9990,0.9726,0.8213,0.6855,0.7465,0.7461,0.7495
SD,0.0002,0.0094,0.0503,0.0490,0.0439,0.0440,0.0438


In [ ]:
rf = top5[1]
tuned_rf = tune_model(rf, fold=5, optimize="F1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9990,0.9750,0.7571,0.6709,0.7114,0.7109,0.7122
1,0.9991,0.9792,0.8429,0.6941,0.7613,0.7608,0.7644
2,0.9990,0.9533,0.8028,0.6706,0.7308,0.7303,0.7332
3,0.9994,0.9824,0.9155,0.7927,0.8497,0.8494,0.8516
4,0.9993,0.9604,0.8169,0.7945,0.8056,0.8052,0.8053
Mean,0.9992,0.9700,0.8270,0.7246,0.7717,0.7713,0.7734
SD,0.0002,0.0113,0.0522,0.0570,0.0503,0.0504,0.0501


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,13:18:49
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Classifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
lgbm = top5[2]
tuned_lgbm = tune_model(lgbm, fold=5, optimize="F1")

In [ ]:
knn = top5[3]
tuned_knn = tune_model(knn, fold=5, optimize="F1")

In [ ]:
dt = top5[4]
tuned_dt = tune_model(dt, fold=5, optimize="F1")

In [ ]:
blended_model = blend_models(estimator_list=[tuned_et, tuned_rf, tuned_lgbm, tuned_knn, tuned_dt])

In [ ]:
predict_model(blended_model)

In [ ]:
plot_model(blended_model, plot="pr")

In [ ]:
plot_model(blended_model, plot="confusion_matrix")

In [ ]:
unseen_predictions = predict_model(blended_model, data=data_unseen)
unseen_predictions.head()

In [ ]:
f1_score = check_metric(unseen_predictions.Class, unseen_predictions.Label, "F1")
Precision = check_metric(unseen_predictions.Class, unseen_predictions.Label, "Precision")
Recall = check_metric(unseen_predictions.Class, unseen_predictions.Label, "Recall")
Accuracy = check_metric(unseen_predictions.Class, unseen_predictions.Label, "Accuracy")

print("F1 score: ", f1_score)
print("Precision: ", Precision)
print("Recall: ", Recall)
print("Accuracy: ", Accuracy)

In [ ]:
save_model(blended_model, "..\models\blended_expt1")